In [ ]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

from pyspark.sql.functions import to_date

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Get:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:12 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1,058 kB]
Get:13 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,776 kB]
Get:14 http://

In [ ]:
# Import packages
from pyspark.sql import SparkSession
#import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
import os

# Get the current working directory
current_dir = os.getcwd()
print("Current directory:", current_dir)

# List the files in the current directory
file_list = os.listdir(current_dir)
print("Files in the directory:", file_list)



Current directory: /content
Files in the directory: ['.config', 'spark-3.4.0-bin-hadoop3.tgz', 'spark-3.4.0-bin-hadoop3', 'heart_2020_cleaned.csv', 'sample_data']


In [ ]:
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder.getOrCreate()

# Read the CSV file into a Spark DataFrame
df = spark.read.csv("heart_2020_cleaned.csv", header=True, inferSchema=True)

# Show the Spark DataFrame
df.show()



+------------+-----+-------+---------------+------+--------------+------------+-----------+------+-----------+-----+--------------------+----------------+---------+---------+------+-------------+----------+
|HeartDisease|  BMI|Smoking|AlcoholDrinking|Stroke|PhysicalHealth|MentalHealth|DiffWalking|   Sex|AgeCategory| Race|            Diabetic|PhysicalActivity|GenHealth|SleepTime|Asthma|KidneyDisease|SkinCancer|
+------------+-----+-------+---------------+------+--------------+------------+-----------+------+-----------+-----+--------------------+----------------+---------+---------+------+-------------+----------+
|          No| 16.6|    Yes|             No|    No|           3.0|        30.0|         No|Female|      55-59|White|                 Yes|             Yes|Very good|      5.0|   Yes|           No|       Yes|
|          No|20.34|     No|             No|   Yes|           0.0|         0.0|         No|Female|80 or older|White|                  No|             Yes|Very good|      7.

In [ ]:
df.printSchema()


root
 |-- HeartDisease: string (nullable = true)
 |-- BMI: double (nullable = true)
 |-- Smoking: string (nullable = true)
 |-- AlcoholDrinking: string (nullable = true)
 |-- Stroke: string (nullable = true)
 |-- PhysicalHealth: double (nullable = true)
 |-- MentalHealth: double (nullable = true)
 |-- DiffWalking: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- AgeCategory: string (nullable = true)
 |-- Race: string (nullable = true)
 |-- Diabetic: string (nullable = true)
 |-- PhysicalActivity: string (nullable = true)
 |-- GenHealth: string (nullable = true)
 |-- SleepTime: double (nullable = true)
 |-- Asthma: string (nullable = true)
 |-- KidneyDisease: string (nullable = true)
 |-- SkinCancer: string (nullable = true)



In [ ]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('heart_disease')

In [ ]:
# calculate the average skin cancer cases per positive heart disease cases
# Create a SparkSession
spark = SparkSession.builder.getOrCreate()

# Read the CSV file into a Spark DataFrame
df = spark.read.csv("heart_2020_cleaned.csv", header=True, inferSchema=True)

# Write the SQL query
avg_df = """
SELECT
  HeartDisease AS HeartDisease,
  SkinCancer AS SkinCancer,
  COUNT(CASE WHEN SkinCancer = 'Yes' THEN 1 ELSE 0 END) AS SkinCancerCount,
  AVG(CASE WHEN SkinCancer = 'Yes' THEN 1 ELSE 0 END) AS AverageSkinCancerCount
FROM heart_disease
GROUP BY HeartDisease, SkinCancer
"""

# Execute the SQL query and show the result
spark.sql(avg_df).show()


+------------+----------+---------------+----------------------+
|HeartDisease|SkinCancer|SkinCancerCount|AverageSkinCancerCount|
+------------+----------+---------------+----------------------+
|         Yes|       Yes|           3975|                   1.0|
|          No|        No|         211392|                   0.0|
|         Yes|        No|          17765|                   0.0|
|          No|       Yes|          19804|                   1.0|
+------------+----------+---------------+----------------------+



SkinCancer 1.56% total.  18.19% of Heart disease patients have skin cancer.

In [ ]:
# calculate the average skin cancer cases per positive heart disease cases

# Write the SQL query
avg_df_2 = """
SELECT
  HeartDisease AS HeartDisease,
  Diabetic AS Diabetic,
  COUNT(CASE WHEN Diabetic = 'Yes' THEN 1 ELSE 0 END) AS DiabeticCount,
  AVG(CASE WHEN Diabetic = 'Yes' THEN 1 ELSE 0 END) AS AverageDiabeticCount
FROM heart_disease
GROUP BY HeartDisease, Diabetic
"""

# Execute the SQL query and show the result
spark.sql(avg_df_2).show()


+------------+--------------------+-------------+--------------------+
|HeartDisease|            Diabetic|DiabeticCount|AverageDiabeticCount|
+------------+--------------------+-------------+--------------------+
|         Yes|                 Yes|         8957|                 1.0|
|          No|Yes (during pregn...|         2451|                 0.0|
|          No|                  No|       252134|                 0.0|
|         Yes|Yes (during pregn...|          108|                 0.0|
|         Yes|                  No|        17519|                 0.0|
|          No|No, borderline di...|         5992|                 0.0|
|          No|                 Yes|        31845|                 1.0|
|         Yes|No, borderline di...|          789|                 0.0|
+------------+--------------------+-------------+--------------------+



Diabetic 2.85% total.  33.12% of Heart Disease patients have Diabetes

In [ ]:
# calculate the average skin cancer cases per positive heart disease cases
# Write the SQL query
avg_df_3 = """
SELECT
  HeartDisease AS HeartDisease,
  KidneyDisease AS KidneyDisease,
  COUNT(CASE WHEN KidneyDisease = 'Yes' THEN 1 ELSE 0 END) AS KidneyDiseaseCount,
  AVG(CASE WHEN KidneyDisease = 'Yes' THEN 1 ELSE 0 END) AS AverageKidneyDiseaseCount
FROM heart_disease
GROUP BY HeartDisease, KidneyDisease
"""

# Execute the SQL query and show the result
spark.sql(avg_df_3).show()

+------------+-------------+------------------+-------------------------+
|HeartDisease|KidneyDisease|KidneyDiseaseCount|AverageKidneyDiseaseCount|
+------------+-------------+------------------+-------------------------+
|         Yes|          Yes|              3455|                      1.0|
|          No|           No|            284098|                      0.0|
|         Yes|           No|             23918|                      0.0|
|          No|          Yes|              8324|                      1.0|
+------------+-------------+------------------+-------------------------+



KidneyDisease 1.08% total.  12.62% of Heart Disaease patients have Kidney Disease

In [ ]:
# calculate the average skin cancer cases per positive heart disease cases
# Write the SQL query
avg_df_4 = """
SELECT
  HeartDisease AS HeartDisease,
  Asthma AS Asthma,
  COUNT(CASE WHEN Asthma = 'Yes' THEN 1 ELSE 0 END) AS AsthmaeCount,
  AVG(CASE WHEN Asthma = 'Yes' THEN 1 ELSE 0 END) AS AverageAsthmaCount
FROM heart_disease
GROUP BY HeartDisease, Asthma
"""

# Execute the SQL query and show the result
spark.sql(avg_df_4).show()

+------------+------+------------+------------------+
|HeartDisease|Asthma|AsthmaeCount|AverageAsthmaCount|
+------------+------+------------+------------------+
|         Yes|   Yes|        4933|               1.0|
|          No|    No|      254483|               0.0|
|         Yes|    No|       22440|               0.0|
|          No|   Yes|       37939|               1.0|
+------------+------+------------+------------------+



Asthma 1.54% total.  18.02% Of Heart Disease patients have Asthma

In [ ]:
#Smoking
# Write the SQL query
avg_df_5 = """
SELECT
  HeartDisease AS HeartDisease,
  Smoking AS Smoking,
  COUNT(CASE WHEN Smoking = 'Yes' THEN 1 ELSE 0 END) AS SmokingCount,
  AVG(CASE WHEN Smoking = 'Yes' THEN 1 ELSE 0 END) AS AverageSmokingCount
FROM heart_disease
GROUP BY HeartDisease, Smoking
"""

# Execute the SQL query and show the result
spark.sql(avg_df_5).show()

+------------+-------+------------+-------------------+
|HeartDisease|Smoking|SmokingCount|AverageSmokingCount|
+------------+-------+------------+-------------------+
|         Yes|    Yes|       16037|                1.0|
|          No|     No|      176551|                0.0|
|         Yes|     No|       11336|                0.0|
|          No|    Yes|      115871|                1.0|
+------------+-------+------------+-------------------+



smoking 5.01% total 58.59% of Heart Disease patients smoke

In [ ]:
#AlcoholDrinking
# Write the SQL query
avg_df_6 = """
SELECT
  HeartDisease AS HeartDisease,
  AlcoholDrinking AS AlcoholDrinking,
  COUNT(CASE WHEN AlcoholDrinking = 'Yes' THEN 1 ELSE 0 END) AS AlcoholDrinkingCount,
  AVG(CASE WHEN AlcoholDrinking = 'Yes' THEN 1 ELSE 0 END) AS AverageAlcoholDrinkingCount
FROM heart_disease
GROUP BY HeartDisease, AlcoholDrinking
"""

# Execute the SQL query and show the result
spark.sql(avg_df_6).show()

+------------+---------------+--------------------+---------------------------+
|HeartDisease|AlcoholDrinking|AlcoholDrinkingCount|AverageAlcoholDrinkingCount|
+------------+---------------+--------------------+---------------------------+
|         Yes|            Yes|                1141|                        1.0|
|          No|             No|              271786|                        0.0|
|         Yes|             No|               26232|                        0.0|
|          No|            Yes|               20636|                        1.0|
+------------+---------------+--------------------+---------------------------+



AlcoholDrinking 0.357% total.  4.178% of Heart Disease patients drink alcohol

In [ ]:
#Stroke
# Write the SQL query
avg_df_7 = """
SELECT
  HeartDisease AS HeartDisease,
  Stroke AS Stroke,
  COUNT(CASE WHEN Stroke = 'Yes' THEN 1 ELSE 0 END) AS StrokeCount,
  AVG(CASE WHEN Stroke = 'Yes' THEN 1 ELSE 0 END) AS AverageStrokeCount
FROM heart_disease
GROUP BY HeartDisease, Stroke
"""

# Execute the SQL query and show the result
spark.sql(avg_df_7).show()

+------------+-------+-----------+------------------+
|HeartDisease|Strokeg|StrokeCount|AverageStrokeCount|
+------------+-------+-----------+------------------+
|         Yes|    Yes|       4389|               1.0|
|          No|     No|     284742|               0.0|
|         Yes|     No|      22984|               0.0|
|          No|    Yes|       7680|               1.0|
+------------+-------+-----------+------------------+



Stroke 1.37% total.  16.03% of Heart Disease patients have had a stroke.

In [ ]:
#DiffWalking
# Write the SQL query
avg_df_8 = """
SELECT
  HeartDisease AS HeartDisease,
  DiffWalking AS DiffWalking,
  COUNT(CASE WHEN DiffWalking = 'Yes' THEN 1 ELSE 0 END) AS DiffWalkingCount,
  AVG(CASE WHEN DiffWalking = 'Yes' THEN 1 ELSE 0 END) AS AverageDiffWalkingCount
FROM heart_disease
GROUP BY HeartDisease, DiffWalking
"""

# Execute the SQL query and show the result
spark.sql(avg_df_8).show()

+------------+-----------+----------------+-----------------------+
|HeartDisease|DiffWalking|DiffWalkingCount|AverageDiffWalkingCount|
+------------+-----------+----------------+-----------------------+
|         Yes|        Yes|           10028|                    1.0|
|          No|         No|          258040|                    0.0|
|         Yes|         No|           17345|                    0.0|
|          No|        Yes|           34382|                    1.0|
+------------+-----------+----------------+-----------------------+



DiffWalking 3.14% total.  36.63% of Heart Disease patients have difficulting walking

In [17]:
# White men over 50
# Write the SQL query
avg_df_combined = """
SELECT
  HeartDisease,
  Sex AS Male,
  COUNT(CASE WHEN Sex = 'Male' THEN 1 ELSE 0 END) AS MaleCount,
  Race AS White,
  COUNT(CASE WHEN Race = 'White' THEN 1 ELSE 0 END) AS RaceCount
FROM heart_disease
WHERE Sex = 'Male' AND AgeCategory IN ('55-59', '60-64', '65-69', '70-74', '75-79', '80 or older')
GROUP BY HeartDisease, Sex, Race


"""
# Execute the SQL query and show the result
spark.sql(avg_df_combined).show()


+------------+----+---------+--------------------+---------+
|HeartDisease|Male|MaleCount|               White|RaceCount|
+------------+----+---------+--------------------+---------+
|         Yes|Male|      298|               Other|      298|
|         Yes|Male|      118|               Asian|      118|
|         Yes|Male|     9979|               White|     9979|
|         Yes|Male|      509|               Black|      509|
|          No|Male|     1949|            Hispanic|     1949|
|          No|Male|      836|               Asian|      836|
|          No|Male|     3380|               Black|     3380|
|          No|Male|    43328|               White|    43328|
|          No|Male|     1338|               Other|     1338|
|          No|Male|      669|American Indian/A...|      669|
|         Yes|Male|      328|            Hispanic|      328|
|         Yes|Male|      156|American Indian/A...|      156|
+------------+----+---------+--------------------+---------+



In [ ]:
#BMI
avg_df_9 = """
SELECT
  HeartDisease AS HeartDisease,
  sum(BMI) AS BMICount
FROM heart_disease
WHERE HeartDisease = 'Yes'
GROUP BY HeartDisease, BMI
"""

spark.sql(avg_df_9).show()


+------------+------------------+
|HeartDisease|          BMICount|
+------------+------------------+
|         Yes|1640.1999999999994|
|         Yes|            323.18|
|         Yes|1261.3900000000003|
|         Yes|1158.2600000000002|
|         Yes|            1274.8|
|         Yes|             425.6|
|         Yes|             32.48|
|         Yes|             82.52|
|         Yes|             51.36|
|         Yes|             24.91|
|         Yes|201.89999999999998|
|         Yes| 5076.400000000007|
|         Yes|202.62000000000003|
|         Yes|            120.69|
|         Yes|            138.48|
|         Yes|            131.13|
|         Yes|             78.86|
|         Yes|             73.78|
|         Yes|           1026.48|
|         Yes|55.260000000000005|
+------------+------------------+
only showing top 20 rows

